# Uninformed Tree Search Without Duplicates Detection
In this lab you are going to implement basic tree search methods without duplicates detection:
- ***BFS (Breadth First Search)***
- ***DFS (Depth First Search)***
- ***DFS-L (Depth First Search with Limited Depth)***
- ***DFID (Depth First Iterative Deepening)***.

Two widespread domains will be considered:
- ***15-puzzle***
- ***Panckakes***. 

For ***15-puzzle*** the code that defines `state` and `get_succesors` is already available. For ***Panckakes*** — you have to code it yourself. All search methods have to be coded by you as well, using the code stubs provided.

Run every cell of the notebook and complete the described tasks. Good luck!

In [2]:
%pip install numpy

  Using cached numpy-1.24.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
You should consider upgrading via the '/home/alex_ch/Desktop/lab01-YourSurnameName/lab1/bin/python3.9 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [895]:
import copy
from typing import List, Callable
import numpy as np

## Gem Puzzle (15-puzzle or n-puzzle)

The ***15-puzzle*** (also called ***Gem Puzzle***, Boss Puzzle, Game of Fifteen, Mystic Square and many others) is a sliding puzzle that consists of a frame of numbered square tiles in random order with one tile missing (the so-called blank tile). 

The size of the puzzle may vary. E.g. if the size of the field is $3\times3$, the puzzle is called the 8-puzzle, and if the size of the field is $4 \times 4$, the puzzle is called the 15-puzzle. 

The task is to place the tiles in order (see the figure below) by sliding them. Indeed you can slide a tile only to a blank one (that's way one can think of actually sliding the blank tile).

Note, that half of the initial configurations for the n-puzzle are impossible to resolve, no matter how many moves are made. See [[Wikipedia](https://en.wikipedia.org/wiki/15_puzzle)] for more info. 

![puzzle](img/gem_puzzle.png)

### Representation of a search state for the Gem Puzzle

Indeed, there may exist many ways to represent a search state for the Gem Puzzle. In this lab we will use a a list of integers as an external encoding of the Gem Puzzle state. This list is assumed to contain numbers from 1 to (*size* * *size*), where *size* is the size of the puzzle. The tile with the number *size* * *size* is a blank tile.

For example, the encoding of the start state of the 8-puzzle depicted above will be \[7,2,4,5,9,6,8,3,1\].

Internally, we will use a 2-dimensional numpy array to store puzzle states. 


In [896]:
import copy
from typing import List, Callable
import numpy as np


class GemPuzzleState:
    """
    Implementing search state (or simply — state) in code
    is a very important first step needed to tackle any search problem.
    The suggested implementation (not the only one possible, obviously)
    of the `GemPuzzleState` is comprised of the following fields:

    Attributes
    ----------
    size : int
        Width of game field.

    tile_matrix : ndarray[int, ndim=2]
        Tile positions represented as a 2d array of integers. This array is
        expected to contain values from `1` to `size * size`. Each integer
        value corresponds to a tile and the position in the array (row and column)
        corresponds to the position of the tile on the game field.
        Tile with the value `size * size` is assumed to represent blank position.

    parent : GemPuzzleState
        Pointer to the parent-state. Parent is a predecessor of the state
        in the search-tree. It is used to reconstrruct a path to that
        state from the start state (root of the search tree).

    blank_pos : ndarray[int, ndim=1]
        Position (row and column) of empty tile in tile_matrix.
        Explicitly storing the position of a blank helps to generate
        successors faster.
    """

    def __init__(self, tile_list: List[int]):
        """
        Constructor. Sets tile positions and some basic checks.

        Parameters
        ----------
        tile_list : List[int]
            Tile positions represented as a list of integers. This list is expected to contain
            values from `1` to `size * size`. Each integer value corresponds to a tile and
            the position in the list (index) corresponds to the position of the tile on the game field.
            Tile with the value `size * size` is assumed to represent blank position.
        """
        self.size = int(len(tile_list) ** 0.5)
        blank_value = self.size ** 2
        if (blank_value != len(tile_list)):
            raise Exception(
                "The tile list must contain the number of elements which \
                is equal to the square of an integer!")

        self.tile_matrix = np.array(
            tile_list, dtype=np.int16).reshape(
            (self.size, self.size))

        # Memorizing the position of a blank tile
        # Technically, there is no need to do so,
        # but it makes to get the successors a bit faster
        blank_poses = np.argwhere(self.tile_matrix == blank_value)
        if (len(blank_poses) != 1):
            raise Exception(
                "State should contains single max value as position to blank tile")
        self.blank_pos = blank_poses[0]
        
        # The parent state (predecessor in the search tree) will be set up by
        # the search algorithm.
        self.parent = None


    def __eq__(self, other):
        """
        Comparing one state with the other state.
        """
        return np.array_equal(self.tile_matrix, other.tile_matrix)


    def __str__(self) -> str:
        """
        String representation of game field for printing
        """
        blank_value = self.size ** 2
        result = str(self.tile_matrix).replace(' [','')\
            .replace('[','').replace(']','').replace(str(blank_value),'_') + '\n'
        return result


### Get Succesors

In [897]:
def get_successors(state: GemPuzzleState) -> List[GemPuzzleState]:
    """
    Implementing `get_successors` function is another important step to tackle any search problem.
    This function is presumed to take a particular search state as the input and to return all 
    possible succesors states. i.e. the ones that result from applying all applicable actions to 
    the input state. In case of GemPuzzle the succesors correspond to the board states resulting 
    from moving blank up/down/left/right. If blank goes out of the field after a move, 
    such successor should obviously be discarded.

    Parameters
    ----------
    state : GemPuzzleState
        The input search state. 

    Returns
    -------
    List[GemPuzzleState]
        List with all possible succesors states for input state.
    """
    delta = np.array([[0, 1], [1, 0], [0, -1], [-1, 0]])
    successors = []
    for d in delta:
        new_pos = state.blank_pos + d   # Computing new column and row for blank tile 
                                        # (corresponding to a particular move encoded via 'd')
        
        # If the new position of a blank is valid (i.e. it is still within the field) then
        # a corresponding sucessor should be added to the succesors' list
        if (0 <= new_pos[0] < state.size) and (0 <= new_pos[1] < state.size):
            new_state = copy.deepcopy(state)
            new_state.tile_matrix[tuple(new_state.blank_pos)] = \
                                        new_state.tile_matrix[tuple(new_pos)] # Moving tile
            new_state.tile_matrix[tuple(new_pos)] = new_state.size ** 2 # Setting blank
            new_state.blank_pos = new_pos
            new_state.parent = state
            successors.append(new_state)

    return successors

### Goal check

In [898]:
def state_is_goal(state: GemPuzzleState):
    """
    Handy function that returns `True` if the input `state` corresponds 
    to the goal one (i.e. all tiles are in their places), and `False` otherwise
    """
    goal_list = np.arange(1, state.size ** 2 + 1).reshape(state.size, state.size)
    return np.array_equal(state.tile_matrix, goal_list)

### Path checking

In [899]:
def check_path(last_state: GemPuzzleState):
    """
    Auxiliary function that takes the `last_state` and checks whether this state is a goal. 
    If yes, it unwinds the path using the backpointers and checks whether each successor is 
    reachable from its predecessor.
    """
    curr = last_state
    if not state_is_goal(curr):
        print("Goal was not reached!")
        return False

    while curr.parent is not None:
        prev = curr.parent
        if (curr not in get_successors(prev)):
            print("Unacceptable step!")
            return False                
        curr = prev
    return True

### Path unwinding
Typically the paths are not stored within a search explicitly, but rather implicitly via the parent pointers (pointing to the predecessor in the search tree). Thus, when we reach the goal state and want to reconstruct the whole path we need to trace the parent pointers back to the root of the tree.

In [900]:
def get_path(last_state: GemPuzzleState):
    """
    This function takes a state `last_state` as an input and 
    returns a path to this state from the root of the tree.
    """
    path = []
    curr = last_state
    while curr is not None:
        path.append(curr)
        curr = curr.parent
    return path

## Automated Tests to Check the Implementations of the Search Algorithms
When you finish implementing search algorithms you need to test them, right? The following functions will help you in that. They take your search algorithm as an input and run it on a single simple test (`simple_test`) and on a series of more involved tests (`massive_test`).

These automated tests assume that the seach function, passed as the input, has the following structure:

`search_function(start_state, *optional arguments*) -> (path_found, last_state)`, where

- `start_state` — initial state
- `*optional arguments*` — additional parameters of the search function (if needed), passed via `*args`
- `path_found` — result of the search, `True` if path was found, `False` otherwise
- `last_state` — last state of path. `None` if path was not found

In [901]:
def simple_test(search_function: Callable, *args):
    """
    Function `simple_test` runs `search_function` on a simple 2 x 2 sliding 
    puzzle instance (encoded as [3, 1, 2, 4]).

    The output is as follows:
    - 'Path is OK' and list of states of path — path was found and it is correct
    - 'Path is not OK' — path was found but it is not correct 
    - 'Path not found' — path was not found 
    - 'Execution error' — an error occurred while executing the `SearchFunction` or path validation function

    Parameters
    ----------
    search_function : Callable
        Implementation of search method.
    """
    start_state = GemPuzzleState([3,1,2,4])
    try:
        result = search_function(start_state, *args)
        curr = result[1]
        if(result[0]):
            if(check_path(curr)):
                print("Path is OK!")
                path = get_path(result[1])
                while len(path) != 0:
                    s = path.pop()
                    print(s)
            else:
                print("Path is not OK")
        else:
            print("Path not found")
    except Exception as e:
        print("Execution error")
        print(e)

In [902]:
def massive_test(search_function: Callable, *args):
    """
    Function `massive_test` runs `search_function` on set of different tasks (stored in `data/task_gem.txt`). 
    Initially this file contains 4 different 2 x 2 sliding puzzle instances and 4 different 3 x 3 
    sliding puzzle instances (you can add more if you want). Each instance starts from a new line and
    is represented as a sequence of integers separated by spaces.

    The output is similar to `simple_test` (however the explicit paths for the solved instances are not displayed):

    - 'Path is OK' and path length — path was found and it is correct
    - 'Path is not OK' — path was found but it is not correct 
    - 'Path not found' — path was not found
    - 'Execution error' — an error occurred  while executing the `search_function` or path validation function

    Parameters
    ----------
    search_function : Callable
        Implementation of search method.
    """
    
    tasks_file = open('data/tasks_gem.txt')
    count = 0
    for l in tasks_file:
        count += 1
        state = list(map(int, l.split()))
        task = GemPuzzleState(state)
        try:
            result = search_function(task, *args)
            curr = result[1]
            if(result[0]):
                if(check_path(curr)):
                    path = get_path(result[1])
                    print("Task:", count, "Path is OK! Path length:", len(path))
                else:
                    print("Task:", count, "Path is not OK!")
            else:
                print("Task:", count,  "Path not found(")
        except Exception as e:
            print("Task:", count, "Execution error")
            print(e)
        

## Search Algorithms Implementation


Recall again here that *dublicate detection must not be coded* in this assignment.

### Breadth-First Search (BFS)

In [903]:
def bfs(start):
    """
    Implementation of Breadth-First Search algorithm.
    """

    path_found = False
    res_state = None
    
    visited = [start]
    queue = [start]
    
    while queue:          
        next_state = queue.pop(0)
        
        if state_is_goal(next_state):
            res_state = next_state
            path_found = True
            return (path_found, res_state)
            
        
        for neighbour in get_successors(next_state):
            if neighbour not in visited:
                visited.append(neighbour)
                queue.append(neighbour)
    
    return (path_found, res_state)


In [904]:
# Test your BFS on simple task
simple_test(bfs)

Path is OK!
3 1
2 _

3 1
_ 2

_ 1
3 2

1 _
3 2

1 2
3 _



In [905]:
# If simple test is OK, you should check your implementation in massive test. 
# The rest of the search algorithms are checked in the same way.
massive_test(bfs)

Task: 1 Path is OK! Path length: 5
Task: 2 Path is OK! Path length: 5
Task: 3 Path is OK! Path length: 3
Task: 4 Path is OK! Path length: 4
Task: 5 Path is OK! Path length: 4
Task: 6 Path is OK! Path length: 6
Task: 7 Path is OK! Path length: 5
Task: 8 Path is OK! Path length: 9


### Depth-First Search (DFS)

In [911]:
from collections import deque
def it_dfs(state):
    """
    Implementation of Depth-First Search algorithm.
    """
    path_found = False
    res_state = None
                
    queue = deque([state])
    while queue:
        next_state = queue.popleft()
        if state_is_goal(next_state):
            res_state = next_state
            return (True, res_state)
           
        for neighbour in get_successors(next_state):
            queue.append(neighbour)

    return (False, res_state)

def dfs(state, visited = None):
    """
    Implementation of Depth-First Search algorithm.
    """
    # print(state, state_is_goal(state))
    if visited is None:
        visited = [state]
    
    if state_is_goal(state):
        return (True, state) 
    

    for neighbour in get_successors(state):
        if neighbour not in visited:
            result = dfs(neighbour, visited=visited)
            if result[0]:
                return result
    return (False, None)


Using DFS, you will most likely encounter the fact that this algorithm overcomes the threshold of recursive calls, after which the execution will interrupted.

Other unpleasant outcomes (e.g. dead kernel) are also possible.

In [912]:
simple_test(dfs)

Path is OK!
3 1
2 _

3 1
_ 2

_ 1
3 2

1 _
3 2

1 2
3 _



In [913]:
simple_test(it_dfs)

Path is OK!
3 1
2 _

3 1
_ 2

_ 1
3 2

1 _
3 2

1 2
3 _



In [914]:
# There is no need to start MassiveTest
massive_test(it_dfs)

Task: 1 Path is OK! Path length: 5
Task: 2 Path is OK! Path length: 5
Task: 3 Path is OK! Path length: 3
Task: 4 Path is OK! Path length: 4
Task: 5 Path is OK! Path length: 4
Task: 6 Path is OK! Path length: 6
Task: 7 Path is OK! Path length: 5
Task: 8 Path is OK! Path length: 9


In [915]:
massive_test(it_dfs)

Task: 1 Path is OK! Path length: 5
Task: 2 Path is OK! Path length: 5
Task: 3 Path is OK! Path length: 3
Task: 4 Path is OK! Path length: 4
Task: 5 Path is OK! Path length: 4
Task: 6 Path is OK! Path length: 6
Task: 7 Path is OK! Path length: 5
Task: 8 Path is OK! Path length: 9


But you can create such a simple task, which can be solved by DFS. So your task is to create an instance of the 8-puzzle that is solvable by DFS and the solution contains *at least 3 moves*. 

In [645]:
def dfs_simple_test(search_function, *args):
    # TODO
    # Insert the 8-puzzle task that is solvable by DPS with at least 3 moves
    # your_tile_list = np.random.permutation([1, 2, 3, 4, 5, 6, 7, 8, 9]).tolist()
    your_tile_list = [1, 2, 3, 4, 5, 6, 9, 7, 8]
    start_state = GemPuzzleState(your_tile_list)
    try:
        result = search_function(start_state, *args)
        curr = result[1]
        if(result[0]):
            if(check_path(curr)):
                print("Path is OK!")
                path = get_path(result[1])

                while len(path) != 0:
                    s = path.pop()
                    print(s)
            else:
                print("Path is not OK")
        else:
            print("Path not found")
    except Exception as e:
        print("Execution error")
        print(e)

In [646]:

dfs_simple_test(dfs)

Path is OK!
1 2 3
4 5 6
_ 7 8

1 2 3
4 5 6
7 _ 8

1 2 3
4 5 6
7 8 _



### Depth-First Search (DFS) with Random Choice

You may try to increase the chance of solving the input problem by randomizing the DFS. One way to do so is to recursively go deeper in the search tree not using the first successor returned by the (deterministic) get_succesors function, but rather by picking a random succesor.

Indeed, this technique does not provide any guarantess and in practice it is likely to fail on numerous instances.

In [639]:
def dfs_random(state, visited = None):
    """
    Implementation of Depth-First Search algorithm.
    """
    
    #OPTIONAL
    
    if visited is None:
        visited = [state]
    
    if state_is_goal(state):
        return (True, state) 
    
    for neighbour in np.random.permutation(get_successors(state)):
        if neighbour not in visited:
            result = dfs(neighbour, visited=visited)
            if result[0]:
                return result
    return (False, None)


In [640]:
for i in range(10):
    simple_test(dfs_random)

Path is OK!
3 1
2 _

3 _
2 1

_ 3
2 1

2 3
_ 1

2 3
1 _

2 _
1 3

_ 2
1 3

1 2
_ 3

1 2
3 _

Path is OK!
3 1
2 _

3 1
_ 2

_ 1
3 2

1 _
3 2

1 2
3 _

Path is OK!
3 1
2 _

3 1
_ 2

_ 1
3 2

1 _
3 2

1 2
3 _

Path is OK!
3 1
2 _

3 1
_ 2

_ 1
3 2

1 _
3 2

1 2
3 _

Path is OK!
3 1
2 _

3 1
_ 2

_ 1
3 2

1 _
3 2

1 2
3 _

Path is OK!
3 1
2 _

3 _
2 1

_ 3
2 1

2 3
_ 1

2 3
1 _

2 _
1 3

_ 2
1 3

1 2
_ 3

1 2
3 _

Path is OK!
3 1
2 _

3 1
_ 2

_ 1
3 2

1 _
3 2

1 2
3 _

Path is OK!
3 1
2 _

3 1
_ 2

_ 1
3 2

1 _
3 2

1 2
3 _

Path is OK!
3 1
2 _

3 1
_ 2

_ 1
3 2

1 _
3 2

1 2
3 _

Path is OK!
3 1
2 _

3 1
_ 2

_ 1
3 2

1 _
3 2

1 2
3 _



### Depth First Search with Limited Depth
One of the way to solve problem of overcoming the threshold of recursive calls is explicitly limit the to depth of the search tree by passing an appropriate parameter `limit` to the search algorithm. The second parameter `depth` is a technical one needed for the implementation. It represents the current depth of the search. Initially (when invoked on the start state of the problem) it is, indeed, equal to 0.

In [632]:
def dfs_l(state, limit, depth, visited = None):
    """
    Implementation of Depth First Search with Limited Depth algorithm
    
    """
    result = (False, None) 
    if visited is None:
        visited = [state]

    if state_is_goal(state) and depth <= limit:
        return (True, state)
    
    if limit == 0:
        if state_is_goal(state):
            return (True, state)
        return result
    
    for neighbour in get_successors(state):
        if neighbour not in visited:
            result = dfs_l(neighbour, limit-1, depth, visited)
            if result[0]:
                return result        
    return result


Let's check this approach with several different limits

In [633]:
simple_test(dfs_l, 3, 0)

Path not found


In [634]:
simple_test(dfs_l, 5, 0)

Path is OK!
3 1
2 _

3 1
_ 2

_ 1
3 2

1 _
3 2

1 2
3 _



In [635]:
simple_test(dfs_l, 10, 0)

Path is OK!
3 1
2 _

3 1
_ 2

_ 1
3 2

1 _
3 2

1 2
3 _



In [636]:
massive_test(dfs_l, 3, 0)

Task: 1 Path not found(
Task: 2 Path not found(
Task: 3 Path is OK! Path length: 3
Task: 4 Path is OK! Path length: 4
Task: 5 Path is OK! Path length: 4
Task: 6 Path not found(
Task: 7 Path not found(
Task: 8 Path not found(


In [637]:
massive_test(dfs_l, 5, 0)

Task: 1 Path is OK! Path length: 5
Task: 2 Path is OK! Path length: 5
Task: 3 Path is OK! Path length: 3
Task: 4 Path is OK! Path length: 4
Task: 5 Path is OK! Path length: 6
Task: 6 Path is OK! Path length: 6
Task: 7 Path is OK! Path length: 5
Task: 8 Path not found(


In [638]:
massive_test(dfs_l, 10, 0)

Task: 1 Path is OK! Path length: 5
Task: 2 Path is OK! Path length: 11
Task: 3 Path is OK! Path length: 3
Task: 4 Path is OK! Path length: 10
Task: 5 Path is OK! Path length: 10
Task: 6 Path is OK! Path length: 10
Task: 7 Path is OK! Path length: 11
Task: 8 Path is OK! Path length: 11


### Depth First Iterative Deepening Search (DFID)
Finally let's sequentially invoke DFS with increasing depth limits. This is called the Depth First Iterative Deepening algorithm. It will inded find a solution if one exists.

In [629]:
def dfid(state):
    """
    Implementation of Iterative-Deepening Depth-First Search
    """
    limit = 1
    visited = None
    while True:
        result = dfs_l(state, limit = limit, depth = 0, visited = visited)
        limit +=1
        if result[0]:
            break
    return result

    #CODE HERE

In [630]:
simple_test(dfid)

Path is OK!
3 1
2 _

3 1
_ 2

_ 1
3 2

1 _
3 2

1 2
3 _



In [631]:
massive_test(dfid)

Task: 1 Path is OK! Path length: 5
Task: 2 Path is OK! Path length: 5
Task: 3 Path is OK! Path length: 3
Task: 4 Path is OK! Path length: 4
Task: 5 Path is OK! Path length: 4
Task: 6 Path is OK! Path length: 6
Task: 7 Path is OK! Path length: 5
Task: 8 Path is OK! Path length: 9


## Pancake Sorting

![Example](img/cat.jpg)

Pancake sorting is the colloquial term for the mathematical problem of sorting a disordered stack of pancakes in order of size when a spatula can be inserted at any point in the stack and used to flip all pancakes above it (See picture below) [[Wikipedia](https://en.wikipedia.org/wiki/Pancake_sorting)].

![Example](img/pancake.png)

### Representation of a state
In this task you should create your own implementation of pancake sorting problem state (and all related funtions) with your own test data. Note, that the interface of the state-class must be the same as for the `GemPuzzleState` thus all the machinery introduced before (e.g. automated tests) will work out-of-the-box. 

In [880]:
class PancakeDish:

    def __init__(self, pancakes: list):
        self.size = len(pancakes)
        blank_value = self.size
        
        self.tile_matrix = np.array(pancakes, dtype=np.int16)
        
        # The parent state (predecessor in the search tree) will be set up by
        # the search algorithm.
        self.parent = None

    def __eq__(self, other):
        return np.array_equal(self.tile_matrix, other.tile_matrix)
        
    def __str__(self):
        base = 3
        result = str()
        for pancake in self.tile_matrix[::-1]:
            result += (base * pancake * f'-{pancake}-').center(30 + 2*base," ") +'\n'
        return result
        
        


In [881]:
def get_successors(state: PancakeDish):
    successors = []
    
    idxs = np.arange(-state.size, -1, dtype = np.int16)
    for idx in idxs:
        new_state = copy.deepcopy(state)
        new_state.tile_matrix[idx:] = new_state.tile_matrix[idx:][::-1]
        
        new_state.parent = state
        # print(new_state.tile_matrix)
        successors.append(new_state)
    return successors

In [882]:
def state_is_goal(state: PancakeDish):
    blank_value = state.size
    return np.array_equal(state.tile_matrix, 
                          np.array(sorted(state.tile_matrix, reverse = True)))

In [884]:
#test
state = PancakeDish([1,2,3,4,5])
print(state)
print(state_is_goal(state))

state = PancakeDish([1,2,3,4,5,9,12])
print(state)
print(state_is_goal(state))

print(get_successors(state))

-5--5--5--5--5--5--5--5--5--5--5--5--5--5--5-
-4--4--4--4--4--4--4--4--4--4--4--4-
    -3--3--3--3--3--3--3--3--3-     
         -2--2--2--2--2--2-         
             -1--1--1-              

False
-12--12--12--12--12--12--12--12--12--12--12--12--12--12--12--12--12--12--12--12--12--12--12--12--12--12--12--12--12--12--12--12--12--12--12--12-
-9--9--9--9--9--9--9--9--9--9--9--9--9--9--9--9--9--9--9--9--9--9--9--9--9--9--9-
-5--5--5--5--5--5--5--5--5--5--5--5--5--5--5-
-4--4--4--4--4--4--4--4--4--4--4--4-
    -3--3--3--3--3--3--3--3--3-     
         -2--2--2--2--2--2-         
             -1--1--1-              

False
[<__main__.PancakeDish object at 0x7f10159534c0>, <__main__.PancakeDish object at 0x7f1015953370>, <__main__.PancakeDish object at 0x7f10159535e0>, <__main__.PancakeDish object at 0x7f1015953f70>, <__main__.PancakeDish object at 0x7f1015953040>, <__main__.PancakeDish object at 0x7f1016e9a2b0>]


In [885]:
def simple_test(search_function: Callable, *args):
    # TODO your simple task
    start_state = PancakeDish([1, 4, 2, 3])

    try:
        result = search_function(start_state, *args)
        curr = result[1]
        if(result[0]):
            if(check_path(curr)):
                print("Path is OK!")
                stack = []
                curr = result[1]
                while curr is not None:
                    stack.append(curr)
                    curr = curr.parent
                while len(stack) != 0:
                    s = stack.pop()
                    print(s)
            else:
                print("Path is not OK")
        else:
            print("Path not found")
    except Exception as e:
        print("Execution error")
        print(e)

In [886]:
def massive_test(search_function: Callable, *args):
    # TODO 
    #Fill the file with at least 8 different tasks
    tasks_file = open('data/tasks_capcake.txt')
    count = 0
    for l in tasks_file:
        count += 1
        state = list(map(int, l.split()))
        task = PancakeDish(state)
        try:
            result = search_function(task, *args)
            curr = result[1]
            if(result[0]):
                if(check_path(curr)):
                    path = get_path(result[1])
                    print("Task:", count, "Path is OK!. Path length:", len(path))
                else:
                    print("Task:", count, "Path is not OK")
            else:
                print("Task:", count, "Path not found")
        except Exception as e:
            print("Task:", count, "Execution error")
            print(e)

## Lets check!

In [887]:
simple_test(bfs)

Path is OK!
    -3--3--3--3--3--3--3--3--3-     
         -2--2--2--2--2--2-         
-4--4--4--4--4--4--4--4--4--4--4--4-
             -1--1--1-              

         -2--2--2--2--2--2-         
    -3--3--3--3--3--3--3--3--3-     
-4--4--4--4--4--4--4--4--4--4--4--4-
             -1--1--1-              

-4--4--4--4--4--4--4--4--4--4--4--4-
    -3--3--3--3--3--3--3--3--3-     
         -2--2--2--2--2--2-         
             -1--1--1-              

             -1--1--1-              
         -2--2--2--2--2--2-         
    -3--3--3--3--3--3--3--3--3-     
-4--4--4--4--4--4--4--4--4--4--4--4-



In [888]:
massive_test(bfs)

Task: 1 Path is OK!. Path length: 5
Task: 2 Path is OK!. Path length: 5
Task: 3 Path is OK!. Path length: 3
Task: 4 Path is OK!. Path length: 6
Task: 5 Path is OK!. Path length: 5
Task: 6 Path is OK!. Path length: 4


In [889]:
simple_test(dfs_l, 2, 0)
simple_test(dfs_l, 5, 0)
simple_test(dfs_l, 10, 0)

Path not found
Path is OK!
    -3--3--3--3--3--3--3--3--3-     
         -2--2--2--2--2--2-         
-4--4--4--4--4--4--4--4--4--4--4--4-
             -1--1--1-              

             -1--1--1-              
-4--4--4--4--4--4--4--4--4--4--4--4-
         -2--2--2--2--2--2-         
    -3--3--3--3--3--3--3--3--3-     

-4--4--4--4--4--4--4--4--4--4--4--4-
             -1--1--1-              
         -2--2--2--2--2--2-         
    -3--3--3--3--3--3--3--3--3-     

    -3--3--3--3--3--3--3--3--3-     
         -2--2--2--2--2--2-         
             -1--1--1-              
-4--4--4--4--4--4--4--4--4--4--4--4-

             -1--1--1-              
         -2--2--2--2--2--2-         
    -3--3--3--3--3--3--3--3--3-     
-4--4--4--4--4--4--4--4--4--4--4--4-

Path is OK!
    -3--3--3--3--3--3--3--3--3-     
         -2--2--2--2--2--2-         
-4--4--4--4--4--4--4--4--4--4--4--4-
             -1--1--1-              

             -1--1--1-              
-4--4--4--4--4--4--4--4--4--4-

In [890]:
massive_test(dfs_l, 2, 0)

Task: 1 Path not found
Task: 2 Path not found
Task: 3 Path is OK!. Path length: 3
Task: 4 Path not found
Task: 5 Path not found
Task: 6 Path not found


In [891]:
massive_test(dfs_l, 5, 0)

Task: 1 Path is OK!. Path length: 5
Task: 2 Path is OK!. Path length: 5
Task: 3 Path is OK!. Path length: 6
Task: 4 Path is OK!. Path length: 6
Task: 5 Path is OK!. Path length: 6
Task: 6 Path is OK!. Path length: 6


In [892]:
massive_test(dfs_l, 10, 0)

Task: 1 Path is OK!. Path length: 11
Task: 2 Path is OK!. Path length: 11
Task: 3 Path is OK!. Path length: 10
Task: 4 Path is OK!. Path length: 10
Task: 5 Path is OK!. Path length: 10
Task: 6 Path is OK!. Path length: 11


In [893]:
simple_test(dfid)

Path is OK!
    -3--3--3--3--3--3--3--3--3-     
         -2--2--2--2--2--2-         
-4--4--4--4--4--4--4--4--4--4--4--4-
             -1--1--1-              

         -2--2--2--2--2--2-         
    -3--3--3--3--3--3--3--3--3-     
-4--4--4--4--4--4--4--4--4--4--4--4-
             -1--1--1-              

-4--4--4--4--4--4--4--4--4--4--4--4-
    -3--3--3--3--3--3--3--3--3-     
         -2--2--2--2--2--2-         
             -1--1--1-              

             -1--1--1-              
         -2--2--2--2--2--2-         
    -3--3--3--3--3--3--3--3--3-     
-4--4--4--4--4--4--4--4--4--4--4--4-



In [894]:
massive_test(dfid)

Task: 1 Path is OK!. Path length: 5
Task: 2 Path is OK!. Path length: 5
Task: 3 Path is OK!. Path length: 3
Task: 4 Path is OK!. Path length: 6
Task: 5 Path is OK!. Path length: 5
Task: 6 Path is OK!. Path length: 4
